<a href="https://colab.research.google.com/github/abolfazlshahsavaryyy/DT/blob/main/DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def Classify_continuous(df:pd.DataFrame,col_name ):

    col=df[col_name]
    min=np.min(col)
    max=np.max(col)
    mean=np.mean([max,min])
    mean_75=np.mean([max,mean])
    mean_25=np.mean([min,mean])
    df.loc[df[col_name]<=min,col_name]=0
    df.loc[(df[col_name]>min )& (df[col_name]<=mean_25),col_name]=1
    df.loc[(df[col_name]>mean_25) & (df[col_name]<=mean),col_name]=2
    df.loc[(df[col_name]>mean) & (df[col_name]<=mean_75),col_name]=3
    df.loc[(df[col_name]>mean_75) & (df[col_name]<=max),col_name]=4
    df.loc[df[col_name]>max ,col_name]=5


In [ ]:
def count_values_columns(df:pd.DataFrame,cols:list):
    for s in cols:
        print(df[s].value_counts())


In [ ]:
def split_X_y(df,y_label="class"):
    y=df[y_label]
    X=df.drop(columns=[y_label])
    return X,y

In [ ]:
def split_train_test_valid(df, random_state=42, test_size=0.2, valid_size=0.2):
    n = len(df)
    np.random.seed(random_state)
    indexes = np.random.permutation(n)


    test_size_df = int(n * test_size)
    valid_size_df = int((n - test_size_df) * valid_size)
    train_size_df = n - (test_size_df + valid_size_df)


    df_train = df.iloc[indexes[:train_size_df]].reset_index(drop=True)
    df_test = df.iloc[indexes[train_size_df:train_size_df + test_size_df]].reset_index(drop=True)
    df_valid = df.iloc[indexes[train_size_df + test_size_df:]].reset_index(drop=True)

    return df_train, df_test, df_valid

In [ ]:
import numpy as np
import pandas as pd

class Node:
    def __init__(self, feature=None, children=None, gini=None, entropy=None, value=None):
        self.feature = feature
        self.children = children or {}
        self.gini = gini
        self.entropy = entropy
        self.value = value

class DecisionTree:
    def __init__(self, criterion="gini", max_depth=None, min_samples_split=2, min_samples_leaf=1, min_impurity_decrease=1e-7):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_impurity_decrease = min_impurity_decrease
        self.root = None

    def fit(self, X, y):
        full_data = pd.DataFrame(X).copy()
        full_data["label"] = y
        self.root = self._build_tree(full_data, depth=0)

    def _build_tree(self, data, depth):
        y = data["label"]
        root_gini = self._gini(y)
        root_entropy = self._entropy(y)
        root_label = self._compute_label(y)

        #base case
        if (self.max_depth is not None and depth >= self.max_depth) or \
           (len(data) < self.min_samples_split) or \
           (len(y.unique()) == 1):
            return Node(value=root_label, gini=root_gini, entropy=root_entropy)

        #find the best feature to split
        best_feature, impurity_decrease = self._best_split_feature(data)


        if best_feature is None or impurity_decrease < self.min_impurity_decrease:
            return Node(value=root_label, gini=root_gini, entropy=root_entropy)

        node = Node(feature=best_feature, gini=root_gini, entropy=root_entropy, children={})

        #split the data based on unique values of the feature
        for category in data[best_feature].unique():
            subset = data[data[best_feature] == category].drop(columns=[best_feature])

            #min samples leaf check
            if len(subset) < self.min_samples_leaf:
                node.children[category] = Node(value=root_label, gini=root_gini, entropy=root_entropy)
            else:
                node.children[category] = self._build_tree(subset, depth + 1)

        return node

    def _best_split_feature(self, data):
        #best score is diffrent for gini and entropy
        #for gini is the weight gini after split
        #for entropy is the entropy before - weight entropy after
        best_feature = None
        best_score = float("inf") if self.criterion == "gini" else 0
        impurity_before_split = self._gini(data["label"]) if self.criterion == "gini" else self._entropy(data["label"])

        for feature in data.columns[:-1]:
            weighted_impurity = self._split_score(data, feature)

            impurity_decrease = impurity_before_split - weighted_impurity


            if (self.criterion == "gini" and weighted_impurity < best_score) or \
               (self.criterion == "entropy" and impurity_decrease > best_score):
                best_feature = feature
                best_score = weighted_impurity if self.criterion == "gini" else impurity_decrease #Information gain

        return best_feature, impurity_decrease if self.criterion == "entropy" else best_score

    def _split_score(self, data, feature):
        total = len(data)
        weighted_impurity = 0

        for category in data[feature].unique():
            subset = data[data[feature] == category]["label"]
            if len(subset) == 0:
                continue
            weight = len(subset) / total

            #check min sample leaf
            if len(subset) < self.min_samples_leaf:
                continue

            impurity = self._gini(subset) if self.criterion == "gini" else self._entropy(subset)
            weighted_impurity += weight * impurity

        return weighted_impurity

    def _gini(self, y):
        proportions = y.value_counts(normalize=True)
        return 1 - np.sum(proportions ** 2)

    def _entropy(self, y):
        proportions = y.value_counts(normalize=True)
        return -np.sum(proportions * np.log2(proportions + 1e-9)) #avoid log(0)

    def predict(self, X):
        return [self._predict_one(row, self.root) for _, row in pd.DataFrame(X).iterrows()]

    def _predict_one(self, row, node):
        if node.value is not None:
            return node.value

        feature_value = row.get(node.feature)

        if feature_value in node.children:
            return self._predict_one(row, node.children[feature_value])
        else:
            return self._compute_label_from_node(node)

    def _compute_label(self, y):
        return y.value_counts().idxmax()

    def _compute_label_from_node(self, node):
        values = [child.value for child in node.children.values() if child.value is not None]
        return max(set(values), key=values.count) if values else None


In [ ]:
def accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

In [ ]:
def change_indexes(i, j, k, t, max_i, max_j, max_k, max_t):
    if t + 1 < max_t:
        return i, j, k, t + 1

    t = 0
    if k + 1 < max_k:
        return i, j, k + 1, t

    k = 0
    if j + 1 < max_j:
        return i, j + 1, k, t

    j = 0
    if i + 1 < max_i:
        return i + 1, j, k, t

    return i, j, k, t


In [ ]:
def hyperparameter_tuning(dict_values, X_train, y_train, X_valid, y_valid):
    result = {}
    max_valid = 0
    best_model = None
    i, j, k, t = 0, 0, 0, 0

    max_i = len(dict_values["max_depth"])
    max_j = len(dict_values["min_samples_leaf"])
    max_k = len(dict_values["min_samples_split"])
    max_t = len(dict_values["criterion"])

    max_diff=0
    for _ in range(max_i * max_j * max_k * max_t):
        max_depth = dict_values["max_depth"][i]
        min_samples_leaf = dict_values["min_samples_leaf"][j]
        min_samples_split = dict_values["min_samples_split"][k]
        criterion = dict_values["criterion"][t]

        model = DecisionTree(max_depth=max_depth, min_samples_leaf=min_samples_leaf,
                             min_samples_split=min_samples_split, criterion=criterion)
        model.fit(X_train, y_train)
        y_pred_valid = model.predict(X_valid)
        y_pred_train = model.predict(X_train)

        train_acc = accuracy(y_train, y_pred_train)
        valid_acc = accuracy(y_valid, y_pred_valid)

        result[f"max_depth:{max_depth}, min_samples_leaf:{min_samples_leaf}, "
               f"min_samples_split:{min_samples_split}, criterion:{criterion}"] = f"train:{train_acc}, valid:{valid_acc}"
        if(abs(train_acc-valid_acc)>max_diff):
            result["max_diff"]=f"max_depth:{max_depth}, min_samples_leaf:{min_samples_leaf}, min_samples_split:{min_samples_split}, criterion:{criterion},train:{train_acc}, valid:{valid_acc}"


        i, j, k, t = change_indexes(i, j, k, t, max_i, max_j, max_k, max_t)

    return result


In [ ]:
def random_delete_samples(df, col, category, random_size_delete=0.5):

    # Get indices of rows where the column has the specified category
    matching_indices = df[df[col] == category].index

    # Calculate the number of rows to delete
    num_to_delete = int(len(matching_indices) * random_size_delete)

    # Randomly select indices to delete
    indices_to_delete = np.random.choice(matching_indices, num_to_delete, replace=False)

    # Drop the selected rows
    df = df.drop(indices_to_delete)

    return df.reset_index(drop=True)

In [ ]:
def delete_extra_samples(df):
    # Filter out rows where 'Attack Type' is 'u2r' or 'r2l'
    df = df[~df["Attack Type"].isin(["u2r", "r2l"])]

    # Reset index after dropping rows
    df = df.reset_index(drop=True)

    return df

In [ ]:
def oversample(df, col, val, rate):

    # Filter rows with the specific value
    df_target = df[df[col] == val]

    # Determine the number of new samples needed
    num_new_samples = int(len(df_target) * (rate - 1))

    # If there's no need to oversample, return the original DataFrame
    if num_new_samples <= 0:
        return df.copy()

    # Randomly sample from the existing target rows
    new_samples = df_target.sample(n=num_new_samples, replace=True, random_state=42)

    # Concatenate with the original DataFrame
    df_oversampled = pd.concat([df, new_samples], ignore_index=True)

    return df_oversampled

In [ ]:
def clean_data(df:pd.DataFrame):

    df=df.dropna()

    df=df.drop(columns=["hot","is_guest_login","same_srv_rate","root_shell","logged_in","is_host_login","num_outbound_cmds","target",
                        "num_failed_logins","land","num_root","wrong_fragment","su_attempted","num_file_creations","is_guest_login",
                        "urgent","is_host_login","num_outbound_cmds","num_access_files","num_shells","num_compromised",
                        "duration","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","dst_host_same_srv_rate",
                        "dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate",
                        "diff_srv_rate","srv_diff_host_rate"])


    map_protocol_type={
        "icmp":0,
        "tcp":1,
        "udp":2
    }
    df["protocol_type"]=df["protocol_type"].map(map_protocol_type)

    map_service = {"ecr_i": 0, "smtp": 1, "http": 2, "private": 3,"domain_u":4}
    df["service"] = df["service"].map(map_service).fillna(5).astype(int)

    map_flag={"SF":0,"S0":1,"REJ":2,"RSTO":4,"RSTR":3}
    df["flag"]=df["flag"].map(map_flag).fillna(5).astype(int)
    col_continuos=["src_bytes","dst_host_same_src_port_rate","count","srv_count",
                   "dst_host_srv_count","dst_host_srv_count","dst_host_count"]
    for s in col_continuos:
        Classify_continuous(df,col_name=s)

    df.loc[(df["dst_bytes"]==0),"dst_bytes"]=0
    df.loc[(df["dst_bytes"]!=0),"dst_bytes"]=1


    df.loc[(df["dst_host_diff_srv_rate"]==0),"dst_host_diff_srv_rate"]=0
    df.loc[(df["dst_host_diff_srv_rate"]!=0),"dst_host_diff_srv_rate"]=1

    df.loc[(df["dst_host_srv_diff_host_rate"]==0),"dst_host_srv_diff_host_rate"]=0
    df.loc[(df["dst_host_srv_diff_host_rate"]!=0),"dst_host_srv_diff_host_rate"]=1


    map_target={
        "normal":0,
        "dos":1,
        "probe":2,


    }
    df["Attack Type"]=df["Attack Type"].map(map_target)
    return df






## start of the project

In [ ]:
df = pd.read_csv("Dataset.csv")
df=df.dropna()
df=random_delete_samples(df,col="Attack Type",category="dos",random_size_delete=0.85)
df=random_delete_samples(df,col="Attack Type",category="normal",random_size_delete=0.4)
df=delete_extra_samples(df)
df=oversample(df,col="Attack Type",val="probe",rate=5)
count_values_columns(df,df.columns.to_list())
df = clean_data(df)


duration
0       129568
1         1593
2          545
3          426
5          343
         ...  
113          1
112          1
66           1
8954         1
2162         1
Name: count, Length: 1666, dtype: int64
protocol_type
tcp     75943
icmp    49066
udp     12612
Name: count, dtype: int64
service
ecr_i      42278
http       37499
private    26100
other      10882
eco_i       6455
           ...  
pop_2         13
ldap          10
urh_i         10
pm_dump        7
tim_i          1
Name: count, Length: 64, dtype: int64
flag
SF        104870
S0         14151
REJ        13832
RSTR        3943
SH           520
RSTO         163
RSTOS0        50
OTH           41
S1            32
S2            15
S3             4
Name: count, dtype: int64
src_bytes
1032    34202
0       32310
520      7809
8        4699
105      4403
        ...  
2589        1
5049        1
1819        1
3250        1
3525        1
Name: count, Length: 2737, dtype: int64
dst_bytes
0        87332
105       2657
147      

In [ ]:
print(count_values_columns(df,df.columns))
train_df,test_df,valid_df=split_train_test_valid(df,random_state=42,test_size=0.2,valid_size=0.2)
x_train,y_train=split_X_y(train_df,y_label="Attack Type")
x_test,y_test=split_X_y(test_df,y_label="Attack Type")
x_valid,y_valid=split_X_y(valid_df,y_label="Attack Type")


protocol_type
1    75943
0    49066
2    12612
Name: count, dtype: int64
service
0    42278
2    37499
3    26100
5    22367
1     5853
4     3524
Name: count, dtype: int64
flag
0    104870
1     14151
2     13832
3      3943
5       662
4       163
Name: count, dtype: int64
src_bytes
1    105308
0     32310
4         3
Name: count, dtype: int64
dst_bytes
0    87332
1    50289
Name: count, dtype: int64
count
1    76250
4    48419
2     9008
3     3938
0        6
Name: count, dtype: int64
srv_count
1    95115
4    41951
2      472
3       77
0        6
Name: count, dtype: int64
dst_host_count
4    98925
1    26009
2     7749
3     4936
0        2
Name: count, dtype: int64
dst_host_srv_count
4    88014
1    40136
2     4945
3     4524
0        2
Name: count, dtype: int64
dst_host_diff_srv_rate
0.0    85837
1.0    51784
Name: count, dtype: int64
dst_host_same_src_port_rate
5.0    94655
0.0    42966
Name: count, dtype: int64
dst_host_srv_diff_host_rate
0.0    100886
1.0     36735
Name: cou

In [ ]:
# hyperparameter={
#     "max_depth":[6,8,10],
#     "min_samples_leaf":[100,150,200],
#     "min_samples_split":[400,600,800],
#     "criterion":["gini","entropy"]
# }
# res=hyperparameter_tuning(hyperparameter,x_train,y_train,x_valid,y_valid)
# res

In [ ]:
#baced on the the res I choose the max_depth=9 and min_samples_leaf=20 and min_sample_split=30 and gini as the criterion
best_model=DecisionTree(max_depth=6,min_samples_leaf=200,min_samples_split=800,criterion="entropy")
best_model.fit(x_train,y_train)
y_pred=best_model.predict(x_test)
y_pred_train=best_model.predict(x_train)
print(f"test: {accuracy(y_pred,y_test)}")
print(f"train: {accuracy(y_train,y_pred_train)}")



test: 0.9694448481325388
train: 0.971661481868344


## Show Tree


In [ ]:
from colorama import Fore
def display_tree(node: Node, prefix=""):
    if not node.children:  # If the node has no children, print it with #
        if(int(node.value)==1):
            print(Fore.GREEN+f"{prefix}{node.value} (normal) #")
        if(int(node.value)==0):
            print(Fore.RED+f"{prefix}{node.value} (dos) #")
        if(int(node.value)==2):
            print(Fore.YELLOW+f"{prefix}{node.value} (prope)#")
    else:
        print(Fore.LIGHTWHITE_EX+f"{prefix}{node.feature} gini:{str(node.gini)[:5]} entropy:{str(node.entropy)[:5]}")
        for edge_label, child in node.children.items():
            display_tree(child, prefix + "  " + str(int(edge_label)) + " → ")

In [ ]:
display_tree(best_model.root)

service gini:0.615 entropy:1.458
  2 → flag gini:0.019 entropy:0.084
  2 →   0 → dst_host_srv_diff_host_rate gini:0.016 entropy:0.072
  2 →   0 →   0 → dst_host_srv_count gini:0.041 entropy:0.147
  2 →   0 →   0 →   4 → dst_host_diff_srv_rate gini:0.031 entropy:0.118
  2 →   0 →   0 →   4 →   1 → 0 (dos) #
  2 →   0 →   0 →   4 →   0 → dst_host_same_src_port_rate gini:0.035 entropy:0.129
  2 →   0 →   0 →   4 →   0 →   0 → 0 (dos) #
  2 →   0 →   0 →   4 →   0 →   5 → 0 (dos) #
  2 →   0 →   0 →   3 → 0 (dos) #
  2 →   0 →   0 →   1 → 0 (dos) #
  2 →   0 →   0 →   2 → 0 (dos) #
  2 →   0 →   0 →   0 → 0 (dos) #
  2 →   0 →   1 → dst_host_diff_srv_rate gini:0.000 entropy:0.004
  2 →   0 →   1 →   0 → 0 (dos) #
  2 →   0 →   1 →   1 → 0 (dos) #
  2 →   2 → dst_host_diff_srv_rate gini:0.010 entropy:0.052
  2 →   2 →   0 → 0 (dos) #
  2 →   2 →   1 → 0 (dos) #
  2 →   4 → 0 (dos) #
  2 →   5 → 0 (dos) #
  2 →   1 → 0 (dos) #
  2 →   3 → 0 (dos) #
  5 → flag gini:0.484 entropy:1.109
  5 →  